In [1]:
import pymorphy2

from spacy.lang import ru
from nltk.stem.snowball import SnowballStemmer 
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop = set(nltk.corpus.stopwords.words('russian'))
Unused_chars=[':',',','.','-','\n','?','!']
stemmer = SnowballStemmer('russian')
morph = pymorphy2.MorphAnalyzer()


def tokenize(file_text):
    tokens = word_tokenize(file_text)
    tokens = [i for i in tokens if (i not in stop)]
    tokens = [i.replace("«", "").replace("»", "") for i in tokens]
    return tokens


def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 2]
    tokens = [get_lemma(token) for token in tokens]
    tokens = [morph.parse(token)[0].normal_form for token in tokens]
    return tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    

def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)




[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nikov\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
#Testing zone, do not launch
import random
import glob
text_data = []
semantic_data = []
wn_data_temp=[]
local_dict={}
TO_DELETE=[]
with open('Lyrics\part2.txt',encoding='utf-8') as f:
    for line in f:
        semantic_data.append(line)
        tokens = prepare_text_for_lda(line)
        if random.random() > .001:
            print(tokens)
            text_data.append(tokens)
            wn_data_temp.append(tokens)
        for token in tokens:
            TO_DELETE.append(token)
            if token in local_dict:
                local_dict[token]+=1
            else:
                local_dict[token]=1



['бел', 'плащ', 'кровавый', 'подбой', 'шаркать', 'кавалерийский', 'походка', 'ранний', 'утром', 'четырнадцать', 'число', 'весенний', 'месяц', 'нисан', 'крытый', 'колоннада', 'два', 'крыло', 'дворец', 'ирод', 'великое', 'выйти', 'прокуратор', 'иудей', 'понтия', 'пилат']
[]
['более', 'свет', 'прокуратор', 'ненавидеть', 'запах', 'розовый', 'масло', 'предвещать', 'нехороший', 'день', 'запах', 'начать', 'преследовать', 'прокуратор', 'рассвет', 'прокуратор', 'казалось', 'розовый', 'запах', 'источать', 'кипарис', 'пальма', 'сад', 'запах', 'кожа', 'конвой', 'примешиваться', 'проклятый', 'розовый', 'струить', 'флигель', 'тыл', 'дворец', 'расположиться', 'прийти', 'прокуратор', 'ершалаим', 'один', 'когорта', 'двенадцать', 'молниеносный', 'легион', 'заносить', 'дымок', 'колоннада', 'верхний', 'площадка', 'сад', 'горьковатый', 'дым', 'свидетельствовать', 'кашевар', 'кентурия', 'начать', 'готовить', 'обед', 'примешиваться', 'жирный', 'розовый', 'дух', 'бог', 'бог', 'наказывать']


In [7]:
#Testing zone, do not launch
import random
import glob
text_data = []
semantic_data = []
wn_data_temp=[]
local_dict={}
TO_DELETE=[]
with open('Lyrics\Незнакомка.txt',encoding='utf-8') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        text_data.append(tokens)

In [219]:
#SOO DA NE 
import random
import glob
text_data = []
semantic_data = []
wn_data_temp=[]
local_dict={}
for filename in glob.glob('Булгаков\*.txt'):
    with open(filename,encoding='utf-8') as f:
        for line in f:
            semantic_data.append(line)
            tokens = prepare_text_for_lda(line)
            if random.random() > .001:
                print(tokens)
                text_data.append(tokens)
                wn_data_temp.append(tokens)
            for token in tokens:
                if token in local_dict:
                    local_dict[token]+=1
                else:
                    local_dict[token]=1

[]
[]
['записка', 'юный', 'врач']
[]
['полотенце', 'петух']
[]
['если', 'человек', 'ездить', 'лошадь', 'глухой', 'просёлочный', 'дорога', 'рассказывать', 'нечего', 'равно', 'понять', 'тот', 'ездить', 'напоминать', 'хотеть']
['сказать', 'коротко', 'сорок', 'верста', 'отделять', 'уездный', 'город', 'грачевка', 'мурьинский', 'больница', 'ехать', 'возница', 'мой', 'ровно', 'сутки', 'курьёзный', 'ровно', 'час', 'сентябрь', '1917', 'год', 'последний', 'лабаз', 'помещаться', 'граница', 'замечательный', 'город', 'грачевка', 'час', 'пять', 'минута', 'сентябрь', 'незабываемый', 'год', 'стоялый', 'битый', 'умирать', 'смякнуть', 'сентябрьский', 'дождик', 'трава', 'двор', 'мурьинский', 'больница', 'стоялый', 'такой', 'вид', 'нога', 'окостенеть', 'настолько', 'смутно', 'двор', 'мысленно', 'перелистывать', 'страница', 'учебник', 'тупо', 'стараться', 'припомнить', 'существовать', 'действительно', 'померещиться', 'вчерашний', 'деревня', 'грабиловка', 'болезнь', 'который', 'человек', 'окостеневать', 'мы

['сейчас', 'помереть', 'угадать', 'мысль', 'шепнуть', 'фельдшер', 'покоситься', 'простыня', 'видимо', 'раздумать', 'жаль', 'кровавить', 'простыня', 'однако', 'несколько', 'секунда', 'прийтись', 'прикрыть', 'лежалый', 'труп', 'умереть', 'голов', 'мой', 'стать', 'светло', 'стеклянный', 'потолок', 'наш', 'далёкий', 'анатомический', 'театр']
['камфара', 'хрипло', 'сказать']
['покорно', 'фельдшер', 'впрыснуть', 'масло']
['неужели', 'умереть', 'отчаянно', 'подумать', 'неужели', 'придется…']
['светлеть', 'мозг', 'всякий', 'учебник', 'совет', 'помощь', 'сообразить', 'уверенность', 'сообразить', 'железный', 'прийтись', 'один', 'жизнь', 'угасать', 'человек', 'делать', 'ампутация', 'человек', 'умереть', 'нож', 'нож', 'умереть', 'ведь', 'кровь', 'десять', 'верста', 'вытечь', 'раздробить', 'нога', 'неизвестно', 'чувствовать', 'слышать', 'молчать', 'почему', 'умирать', 'сказать', 'безумный', 'отец']
['готовить', 'ампутация', 'сказать', 'фельдшер', 'чужое', 'голос']
['акушерка', 'посмотреть', 'дико',

['…совершенно', 'воспрещаться', 'пытаться', 'проникнуть', 'ножка', 'вдоль', 'спинка', 'плода…']
[]
['принять', 'сведение']
[]
['…захватывание', 'верхний', 'ножка', 'следовать', 'считать', 'ошибка', 'легко', 'получиться', 'осевой', 'перекручивание', 'плод', 'который', 'дать', 'повод', 'тяжёлый', 'вколачивание', 'плод', 'вследствие', 'самый', 'печальный', 'последствиям…']
[]
['печальный', 'последствие', 'немного', 'неопределённый', 'какой', 'внушительный', 'слово', 'дульцевский', 'женщина', 'остаться', 'вдовец', 'вытереть', 'испарина', 'собраться', 'силой', 'миновать', 'страшный', 'место', 'постараться', 'запомнить', 'самый', 'существенный', 'собственно', 'должный', 'делать', 'вводить', 'рука', 'пробегать', 'чёрный', 'строчка', 'время', 'наталкиваться', 'новое', 'страшный', 'вещий', 'бить', 'глаз']
[]
['…ввид', 'огромный', 'опасность', 'разрыва…', 'внутренний', 'комбинированный', 'поворот', 'представлять', 'операция', 'который', 'должный', 'отнести', 'опасный', 'матерь', 'акушерский', 'о

['войти', 'малый', 'операционный', 'сквозь', 'завеса', 'увидать', 'блестящий', 'инструмент', 'ослепительный', 'лампа', 'клеенку…', 'последний', 'выйти', 'матерь', 'который', 'девочка', 'вырвать', 'услыхать', 'лишь', 'хриплый', 'голос', 'который', 'говорить', 'муж', 'город', 'прийти', 'узнать', 'наделать', 'убить']
['убить', 'повторить', 'бабка', 'глядеть', 'ужас']
['операционный', 'пускать', 'приказать']
['остаться', 'один', 'операционный', 'персонал', 'лидка', 'девочка', 'голенький', 'сидеть', 'стол', 'беззвучно', 'плакать', 'повалить', 'стол', 'прижать', 'горло', 'вымылить', 'смазать', 'йод', 'взять', 'подумать', 'делать', 'быть', 'очень', 'тихо', 'операционный', 'взять', 'провести', 'вертикально', 'черта', 'пухлый', 'белый', 'горло', 'выступить', 'один', 'капля', 'кровь', 'два', 'провести', 'нож', 'бела', 'полоска', 'который', 'выступить', 'раздаться', 'кожа', 'опять', 'кровинка', 'медленно', 'стараться', 'вспомнить', 'какой', 'рисунок', 'атлас', 'стать', 'помощь', 'тупой', 'зонд', 

['загреметь', 'перелеска', 'вьюга', 'свистнуть', 'сбоку', 'сыпнула…', 'я', 'начать', 'качать', 'качать', 'качало…', 'пока', 'оказаться', 'сандуновский', 'баня', 'москва', 'прямо', 'шуба', 'раздевальня', 'испарина', 'покрыть', 'затем', 'загореться', 'факел', 'напустить', 'холод', 'открыть', 'глаз', 'увидеть', 'сиять', 'кровавый', 'шлем', 'подумать', 'пожар…', 'затем', 'очнуться', 'понять', 'привезти', 'порог', 'белый', 'здание', 'колонна', 'видимо', 'время', 'николай', 'глубокий', 'тьма', 'кругом', 'встретить', 'пожарный', 'пламя', 'танцевать', 'голова', 'извлечь', 'щель', 'шуба', 'часы', 'увидеть', 'пять', 'ехать', 'стать', 'половина']
['лошадь', 'обратно', 'дать', 'сказать']
['слушать', 'ответить', 'возница']
['полусонный', 'мокрый', 'компресс', 'кожаный', 'куртка', 'войти', 'сени', 'сбоку', 'ударить', 'свет', 'лампа', 'полоса', 'лечь', 'крашеный', 'выбежать', 'светловолосый', 'юный', 'человек', 'затравить', 'глаз', 'брюки', 'свежезаутюжить', 'складка', 'белый', 'галстук', 'чёрный', '

...

['притворяться', 'грозно', 'сказать', 'иван', 'почувствовать', 'холод', 'ложечка', 'прекрасно', 'говорить', 'по-русски', 'немец', 'профессор', 'убийца', 'шпион', 'документ', 'яростно', 'крикнуть', 'иван']
[]
['загадочный', 'профессор', 'брезгливый', 'скривить', 'кривая', 'пожать', 'плечо']
[]
['гражданин', 'встрять', 'мерзкий', 'регент', 'волновать', 'интурист', 'строжайше', 'спроситься', 'подозрительный', 'профессор', 'сделать', 'надменный', 'лицо', 'повернуться', 'пойти', 'иван', 'прочь']
[]
['иван', 'почувствовать', 'теряться', 'задыхаться', 'обратиться', 'регент']
[]
['гражданин', 'помочь', 'задержать', 'преступник', 'обязать', 'сделать']
[]
['регент', 'чрезвычайно', 'оживиться', 'вскочить', 'заорать']
[]
['твой', 'преступник', 'иностранный', 'преступник', 'глаз', 'регент', 'радостно', 'заиграть', 'ежели', 'преступник', 'один', 'долг', 'следовать', 'кричать', 'караул', 'уйти', 'давать', 'вместе', 'разом', 'регент', 'разинуть', 'пасть']
[]
['растеряться', 'иван', 'послушаться', 'шут

['вопрос', 'дорогой', 'фок', 'арчибальд', 'арчибальд', 'шепнуть', 'сегодня', 'быть', 'порционный', 'судачок', 'натюрель', 'виртуозный', 'штука']
[]
['уметь', 'жить', 'амвросий', 'вздох', 'отвечать', 'тощий', 'запустить', 'карбункул', 'фок', 'румяногубый', 'гигант', 'золотистоволосый', 'пышнощёкий', 'амвросий-поэт']
[]
['никакой', 'умение', 'особенный', 'нету', 'возражать', 'амвросий', 'обыкновенный', 'желание', 'жить', 'по-человечески', 'хотеть', 'сказать', 'фок', 'судачок', 'встретить', 'колизей', 'колизей', 'порция', 'судачок', 'стоить', 'тринадцать', 'рубль', 'пятнадцать', 'копейка', 'пять', 'пятьдесят', 'кроме', 'колизей', 'судачок', 'третьедневочный', 'кроме', 'гарантия', 'получить', 'колизей', 'виноградный', 'кисть', 'морда', 'первое', 'попасть', 'молодой', 'человек', 'ворваться', 'театральный', 'проезд', 'категорически', 'против', 'колизей', 'греметь', 'весь', 'бульвар', 'гастроном', 'амвросий', 'уговаривать', 'фок']
[]
['уговаривать', 'амвросий', 'пищать', 'фок', 'дом', 'поужин

['товарищ', 'бездомный', 'заговорить', 'лицо', 'юбилейный', 'голос', 'успокоиться', 'расстроить', 'смерть', 'весь', 'мы', 'любимый', 'михаил', 'александр', 'просто', 'миша', 'берлиоз', 'прекрасно', 'понимать', 'нужный', 'покоить', 'сейчас', 'товарищ', 'проводить', 'постель', 'забыться']
[]
['оскалиться', 'перебить', 'иван', 'понимать', 'поймать', 'профессор', 'лезть', 'свой', 'глупость', 'кретин']
[]
['товарищ', 'бездомный', 'помиловать', 'ответить', 'лицо', 'краснеть', 'пятиться', 'раскаиваться', 'ввязаться', 'дело']
[]
['кто-кто', 'помиловать', 'тихий', 'ненависть', 'сказать', 'иван', 'николай']
[]
['судорога', 'исказить', 'лицо', 'быстро', 'переложить', 'свеча', 'правый', 'рука', 'левый', 'широко', 'размахнуться', 'ударить', 'участливый', 'лицо']
[]
['догадаться', 'броситься', 'иван', 'броситься', 'свеча', 'погаснуть', 'очки', 'соскочить', 'лицо', 'мгновенно', 'растоптать', 'иван', 'испустить', 'страшный', 'боев', 'вопль', 'слышный', 'общий', 'соблазн', 'бульвар', 'начать', 'защищат

['отравить', 'взрыв', 'неврастения', 'поэт', 'покачнуться', 'перестать', 'трястись', 'рюхин', 'поднять', 'голова', 'увидеть', 'москва', 'москва', 'рассвет', 'облако', 'подсветить', 'золото', 'грузовик', 'стоить', 'застрять', 'колонна', 'другой', 'машина', 'поворот', 'бульвар', 'близехонький', 'стоить', 'постамент', 'металлический', 'человек', 'наклонить', 'голова', 'безразличный', 'смотреть', 'бульвар']
[]
['какой-то', 'странный', 'мысль', 'хлынуть', 'голова', 'заболеть', 'поэт', 'пример', 'настоящий', 'удачливость', 'рюхин', 'встать', 'весь', 'рост', 'платформа', 'грузовик', 'рука', 'поднять', 'нападать', 'зачем-то', 'никто', 'трогать', 'чугунный', 'человек', 'сделать', 'жизнь', 'случиться', 'польза', 'обращаться', 'слава', 'сделать', 'понимать', 'что-нибудь', 'особенный', 'этот', 'слово', 'буря', 'мгла', 'понимать', 'повезти', 'повезти', 'ядовито', 'заключить', 'рюхин', 'почувствовать', 'грузовик', 'шевельнуться', 'стрелять', 'стрелять', 'белогвардеец', 'раздробить', 'бедро', 'обеспе

KeyboardInterrupt: 

# Начало основного куска кода

In [8]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')
import gensim
NUM_TOPICS = 3
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=40)
ldamodel.save('model5.gensim')

In [9]:
topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.022*"душа" + 0.022*"мой" + 0.022*"берег" + 0.022*"раздаваться" + 0.022*"дух" + 0.022*"очаровать" + 0.012*"над" + 0.012*"сокровище" + 0.012*"лежать" + 0.012*"канава"')
(1, '0.027*"пьяный" + 0.026*"вечер" + 0.015*"терпкий" + 0.015*"мыть" + 0.015*"чуть" + 0.015*"крендель" + 0.015*"золотиться" + 0.015*"булочный" + 0.015*"медленно" + 0.015*"пройти"')
(2, '0.023*"каждый" + 0.022*"вечер" + 0.022*"перо" + 0.022*"шёлк" + 0.013*"глухой" + 0.013*"окно" + 0.013*"над" + 0.013*"стан" + 0.013*"схватить" + 0.013*"девичий"')


In [55]:
import numpy as np
from polyglot.text import Text as T
from polyglot.detect import Detector

line_list=[]
big_text=''
for string in semantic_data:
    big_text+=string
test_text = T(big_text)
print(test_text.polarity)
for w in test_text.words:
    print("{:<16}{:>2}".format(w, w.polarity))
for s in test_text.sentences:
    print(s,' ', s.polarity)
print('ready')


0.0
Опять            0
,                0
как              1
в                0
годы             0
золотые          0
,                0
Три              0
стертых          0
треплются        0
шлеи             0
,                0
И                0
вязнут           0
спицы            0
росписные        0
В                0
расхлябанные     0
колеи            0
.                0
.                0
.                0
Россия           0
,                0
нищая            0
Россия           0
,                0
Мне              0
избы             0
серые            0
твои             0
,                0
Твои             0
мне              0
песни            0
ветровые         0
,                0
-                0
Как              0
слезы            0
первые           0
любви            0
!                0
Тебя             0
жалеть           0
я                0
не               0
умею             0
И                0
крест            0
свой             0
бережно          0
несу    

# Парсер сайта на символы

In [2]:
#Поиск всех ссылок в ссылках
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
from urllib.parse import unquote
from urllib.error import URLError, HTTPError
import re

def get_start_page(target):
    return "http://www.symbolarium.ru/index.php/{0}".format(quote(target))
def get_page_url(target):
    return "http://www.symbolarium.ru{0}".format(quote(target))

html_page = urllib.request.urlopen(get_start_page("Символика"))
soup = BeautifulSoup(html_page)
links = []
print('start')
#for link in soup.findAll('a', attrs={'href': re.compile("^http://")}):
    #links.append(link.get('href'))
for link in soup.find_all('a', href=True):
    links.append(link.get('href'))
symbol_links=[]
for link in links:
    #print('Это исходный ', unquote(link))
    try:
        SymbolFile = urllib.request.urlopen(get_page_url(unquote(link)))        
    except URLError as e:
        continue
    else:
        SymbolHtml = SymbolFile.read()
        SymbolFile.close()
        soup = BeautifulSoup(SymbolHtml)
        for link in soup.find_all('a', href=True):
            symbol_links.append(link.get('href'))
        
        
print('Ready')      





start
Ready


In [9]:
#Обработка содержимого всех страниц
tag_list=['<p>','</p>','<br/>','/','//']
useless_tag=['Служебная','Категория','Энциклопедия_знаков_и_символов','//','/']
print('Обработка всех символов')
count=0
link_count=len(symbol_links)
for link in symbol_links:
    count+=1
    try:
        SymbolFile = urllib.request.urlopen(get_page_url(unquote(link)))        
    except URLError as e:
        continue
    else:
        text_link=unquote(link)
        SymbolHtml = SymbolFile.read()
        SymbolFile.close()
        soup = BeautifulSoup(SymbolHtml)
        '''
        text=soup.findAll("p")
        file_text=str(text)
        for tag in tag_list:
            file_text=file_text.replace(tag,'')
        file_name=unquote(link).replace('/index.php/','')
        invalid_tags=0
        for tag in useless_tag:
            if tag in file_name:
                invalid_tags+=1
        if (invalid_tags>0):
            continue
        '''
        file_text=soup.get_text()
        with open('Test\\'+file_name+'.txt','w',encoding='utf-8') as f:
            f.write(file_text)
            #f.write('\n end_symbol \n')
        print('Обработано {} страниц из {}'.format(count,link_count))
print('ready')

Обработка всех символов
Обработано 4 страниц из 8686
Обработано 5 страниц из 8686
Обработано 6 страниц из 8686
Обработано 7 страниц из 8686
Обработано 8 страниц из 8686
Обработано 9 страниц из 8686
Обработано 10 страниц из 8686
Обработано 11 страниц из 8686
Обработано 12 страниц из 8686
Обработано 13 страниц из 8686
Обработано 14 страниц из 8686
Обработано 15 страниц из 8686
Обработано 16 страниц из 8686
Обработано 17 страниц из 8686
Обработано 18 страниц из 8686
Обработано 19 страниц из 8686
Обработано 20 страниц из 8686
Обработано 21 страниц из 8686
Обработано 22 страниц из 8686
Обработано 23 страниц из 8686
Обработано 24 страниц из 8686
Обработано 25 страниц из 8686
Обработано 26 страниц из 8686
Обработано 27 страниц из 8686
Обработано 28 страниц из 8686
Обработано 29 страниц из 8686
Обработано 30 страниц из 8686
Обработано 31 страниц из 8686
Обработано 32 страниц из 8686
Обработано 33 страниц из 8686
Обработано 34 страниц из 8686
Обработано 35 страниц из 8686
Обработано 36 страниц 

Обработано 359 страниц из 8686
Обработано 360 страниц из 8686
Обработано 361 страниц из 8686
Обработано 362 страниц из 8686
Обработано 363 страниц из 8686
Обработано 364 страниц из 8686
Обработано 365 страниц из 8686
Обработано 366 страниц из 8686
Обработано 367 страниц из 8686
Обработано 368 страниц из 8686
Обработано 369 страниц из 8686
Обработано 370 страниц из 8686
Обработано 371 страниц из 8686
Обработано 372 страниц из 8686
Обработано 373 страниц из 8686
Обработано 374 страниц из 8686
Обработано 387 страниц из 8686
Обработано 388 страниц из 8686
Обработано 421 страниц из 8686
Обработано 422 страниц из 8686
Обработано 423 страниц из 8686
Обработано 424 страниц из 8686
Обработано 425 страниц из 8686
Обработано 426 страниц из 8686
Обработано 427 страниц из 8686
Обработано 428 страниц из 8686
Обработано 429 страниц из 8686
Обработано 430 страниц из 8686
Обработано 431 страниц из 8686
Обработано 432 страниц из 8686
Обработано 433 страниц из 8686
Обработано 434 страниц из 8686
Обработа

Обработано 800 страниц из 8686
Обработано 801 страниц из 8686
Обработано 802 страниц из 8686
Обработано 803 страниц из 8686
Обработано 804 страниц из 8686
Обработано 805 страниц из 8686
Обработано 806 страниц из 8686
Обработано 807 страниц из 8686
Обработано 808 страниц из 8686
Обработано 809 страниц из 8686
Обработано 810 страниц из 8686
Обработано 811 страниц из 8686
Обработано 812 страниц из 8686
Обработано 813 страниц из 8686
Обработано 814 страниц из 8686
Обработано 815 страниц из 8686
Обработано 816 страниц из 8686
Обработано 817 страниц из 8686
Обработано 818 страниц из 8686
Обработано 819 страниц из 8686
Обработано 820 страниц из 8686
Обработано 821 страниц из 8686
Обработано 822 страниц из 8686
Обработано 823 страниц из 8686
Обработано 824 страниц из 8686
Обработано 825 страниц из 8686
Обработано 826 страниц из 8686
Обработано 827 страниц из 8686
Обработано 828 страниц из 8686
Обработано 829 страниц из 8686
Обработано 830 страниц из 8686
Обработано 831 страниц из 8686
Обработа

Обработано 1193 страниц из 8686
Обработано 1194 страниц из 8686
Обработано 1195 страниц из 8686
Обработано 1196 страниц из 8686
Обработано 1197 страниц из 8686
Обработано 1198 страниц из 8686
Обработано 1199 страниц из 8686
Обработано 1200 страниц из 8686
Обработано 1201 страниц из 8686
Обработано 1202 страниц из 8686
Обработано 1203 страниц из 8686
Обработано 1204 страниц из 8686
Обработано 1205 страниц из 8686
Обработано 1206 страниц из 8686
Обработано 1207 страниц из 8686
Обработано 1208 страниц из 8686
Обработано 1209 страниц из 8686
Обработано 1210 страниц из 8686
Обработано 1211 страниц из 8686
Обработано 1212 страниц из 8686
Обработано 1213 страниц из 8686
Обработано 1214 страниц из 8686
Обработано 1215 страниц из 8686
Обработано 1216 страниц из 8686
Обработано 1217 страниц из 8686
Обработано 1218 страниц из 8686
Обработано 1219 страниц из 8686
Обработано 1220 страниц из 8686
Обработано 1221 страниц из 8686
Обработано 1222 страниц из 8686
Обработано 1223 страниц из 8686
Обработа

Обработано 1582 страниц из 8686
Обработано 1583 страниц из 8686
Обработано 1584 страниц из 8686
Обработано 1585 страниц из 8686
Обработано 1586 страниц из 8686
Обработано 1599 страниц из 8686
Обработано 1600 страниц из 8686
Обработано 1633 страниц из 8686
Обработано 1634 страниц из 8686
Обработано 1635 страниц из 8686
Обработано 1636 страниц из 8686
Обработано 1637 страниц из 8686
Обработано 1638 страниц из 8686
Обработано 1639 страниц из 8686
Обработано 1640 страниц из 8686
Обработано 1641 страниц из 8686
Обработано 1642 страниц из 8686
Обработано 1643 страниц из 8686
Обработано 1644 страниц из 8686
Обработано 1645 страниц из 8686
Обработано 1646 страниц из 8686
Обработано 1647 страниц из 8686
Обработано 1648 страниц из 8686
Обработано 1649 страниц из 8686
Обработано 1650 страниц из 8686
Обработано 1651 страниц из 8686
Обработано 1652 страниц из 8686
Обработано 1653 страниц из 8686
Обработано 1654 страниц из 8686
Обработано 1667 страниц из 8686
Обработано 1668 страниц из 8686
Обработа

...

Обработано 5520 страниц из 8686
Обработано 5521 страниц из 8686
Обработано 5522 страниц из 8686
Обработано 5523 страниц из 8686
Обработано 5524 страниц из 8686
Обработано 5525 страниц из 8686
Обработано 5526 страниц из 8686
Обработано 5527 страниц из 8686
Обработано 5528 страниц из 8686
Обработано 5529 страниц из 8686
Обработано 5530 страниц из 8686
Обработано 5531 страниц из 8686
Обработано 5532 страниц из 8686
Обработано 5533 страниц из 8686
Обработано 5534 страниц из 8686
Обработано 5535 страниц из 8686
Обработано 5536 страниц из 8686
Обработано 5537 страниц из 8686
Обработано 5538 страниц из 8686
Обработано 5539 страниц из 8686
Обработано 5540 страниц из 8686
Обработано 5541 страниц из 8686
Обработано 5542 страниц из 8686
Обработано 5543 страниц из 8686
Обработано 5544 страниц из 8686
Обработано 5545 страниц из 8686
Обработано 5546 страниц из 8686
Обработано 5547 страниц из 8686
Обработано 5548 страниц из 8686
Обработано 5549 страниц из 8686
Обработано 5550 страниц из 8686
Обработа

Обработано 6601 страниц из 8686
Обработано 6602 страниц из 8686
Обработано 6603 страниц из 8686
Обработано 6604 страниц из 8686
Обработано 6605 страниц из 8686
Обработано 6606 страниц из 8686
Обработано 6607 страниц из 8686
Обработано 6608 страниц из 8686
Обработано 6609 страниц из 8686
Обработано 6610 страниц из 8686
Обработано 6611 страниц из 8686
Обработано 6612 страниц из 8686
Обработано 6613 страниц из 8686
Обработано 6614 страниц из 8686
Обработано 6615 страниц из 8686
Обработано 6616 страниц из 8686
Обработано 6617 страниц из 8686
Обработано 6618 страниц из 8686
Обработано 6619 страниц из 8686
Обработано 6620 страниц из 8686
Обработано 6621 страниц из 8686
Обработано 6622 страниц из 8686
Обработано 6623 страниц из 8686
Обработано 6624 страниц из 8686
Обработано 6625 страниц из 8686
Обработано 6626 страниц из 8686
Обработано 6627 страниц из 8686
Обработано 6628 страниц из 8686
Обработано 6629 страниц из 8686
Обработано 6630 страниц из 8686
Обработано 6631 страниц из 8686
Обработа

Обработано 7283 страниц из 8686
Обработано 7284 страниц из 8686
Обработано 7285 страниц из 8686
Обработано 7286 страниц из 8686
Обработано 7287 страниц из 8686
Обработано 7288 страниц из 8686
Обработано 7289 страниц из 8686
Обработано 7290 страниц из 8686
Обработано 7291 страниц из 8686
Обработано 7292 страниц из 8686
Обработано 7293 страниц из 8686
Обработано 7294 страниц из 8686
Обработано 7295 страниц из 8686
Обработано 7296 страниц из 8686
Обработано 7297 страниц из 8686
Обработано 7298 страниц из 8686
Обработано 7299 страниц из 8686
Обработано 7300 страниц из 8686
Обработано 7301 страниц из 8686
Обработано 7302 страниц из 8686
Обработано 7303 страниц из 8686
Обработано 7304 страниц из 8686
Обработано 7305 страниц из 8686
Обработано 7306 страниц из 8686
Обработано 7307 страниц из 8686
Обработано 7308 страниц из 8686
Обработано 7309 страниц из 8686
Обработано 7310 страниц из 8686
Обработано 7311 страниц из 8686
Обработано 7312 страниц из 8686
Обработано 7313 страниц из 8686
Обработа

Обработано 7616 страниц из 8686
Обработано 7617 страниц из 8686
Обработано 7618 страниц из 8686
Обработано 7619 страниц из 8686
Обработано 7620 страниц из 8686
Обработано 7621 страниц из 8686
Обработано 7622 страниц из 8686
Обработано 7623 страниц из 8686
Обработано 7624 страниц из 8686
Обработано 7638 страниц из 8686
Обработано 7639 страниц из 8686
Обработано 7673 страниц из 8686
Обработано 7674 страниц из 8686
Обработано 7675 страниц из 8686
Обработано 7676 страниц из 8686
Обработано 7677 страниц из 8686
Обработано 7678 страниц из 8686
Обработано 7679 страниц из 8686
Обработано 7680 страниц из 8686
Обработано 7681 страниц из 8686
Обработано 7682 страниц из 8686
Обработано 7683 страниц из 8686
Обработано 7684 страниц из 8686
Обработано 7685 страниц из 8686
Обработано 7686 страниц из 8686
Обработано 7687 страниц из 8686
Обработано 7688 страниц из 8686
Обработано 7689 страниц из 8686
Обработано 7690 страниц из 8686
Обработано 7691 страниц из 8686
Обработано 7692 страниц из 8686
Обработа

Обработано 8009 страниц из 8686
Обработано 8010 страниц из 8686
Обработано 8011 страниц из 8686
Обработано 8012 страниц из 8686
Обработано 8013 страниц из 8686
Обработано 8014 страниц из 8686
Обработано 8015 страниц из 8686
Обработано 8016 страниц из 8686
Обработано 8017 страниц из 8686
Обработано 8018 страниц из 8686
Обработано 8019 страниц из 8686
Обработано 8020 страниц из 8686
Обработано 8021 страниц из 8686
Обработано 8022 страниц из 8686
Обработано 8023 страниц из 8686
Обработано 8024 страниц из 8686
Обработано 8025 страниц из 8686
Обработано 8026 страниц из 8686
Обработано 8027 страниц из 8686
Обработано 8028 страниц из 8686
Обработано 8029 страниц из 8686
Обработано 8030 страниц из 8686
Обработано 8031 страниц из 8686
Обработано 8032 страниц из 8686
Обработано 8033 страниц из 8686
Обработано 8034 страниц из 8686
Обработано 8035 страниц из 8686
Обработано 8036 страниц из 8686
Обработано 8037 страниц из 8686
Обработано 8038 страниц из 8686
Обработано 8039 страниц из 8686
Обработа

In [2]:
for link in symbol_links:
    print(unquote(link))

//www.mediawiki.org/wiki/Special:MyLanguage/Help:Categories
#mw-head
#p-search
/index.php/Аллегория
/index.php/Аномалия
/index.php/Анонимность
/index.php/Антропогонические_мифы
/index.php/Арматюры
/index.php/Аромат
/index.php/Архетип
/index.php/Атрибуты
/index.php/Басня
/index.php/Близнечные_мифы
/index.php/Богиня-мать
/index.php/Божественный_посланник
/index.php/Вестники_богов
/index.php/Время_мифическое
/index.php/Геометрические_символы
/index.php/Двуполые_существа
/index.php/Демиург
/index.php/Древо_Жизни
/index.php/Древо_мировое
/index.php/Древо_познания
/index.php/Дуалистические_мифы
/index.php/Дуалистические_символы
/index.php/Дух,_духи
/index.php/Живопись
/index.php/Загадка
/index.php/Заговоры
/index.php/Запах
/index.php/Знак
/index.php/Идиллия
/index.php/Иероглифика
/index.php/Изобразительное_искусство_и_мифология
/index.php/Инверсия
/index.php/Индивидуация
/index.php/История_и_мифы
/index.php/Катастрофа
/index.php/Конец_Света
/index.php/Конфуцианство
/index.php/Космическое_яйц

/index.php/Подсолнечник
/index.php/Полынь
/index.php/Роза
/index.php/Розмарин
/index.php/Сад
/index.php/Слива
/index.php/Тростник
/index.php/Тубероза
/index.php/Тысячелистник
/index.php/Тюльпан
/index.php/Фиалка
/index.php/Хризантема
/index.php/Хризотамн
/index.php/Цикламен
/index.php/Чертополох
/index.php/Шафран
/index.php/Язык_цветов
http://www.symbolarium.ru/index.php?title=Категория:Растения_–_Цветы&oldid=78441
/index.php/Служебная:Категории
/index.php/Категория:Растения
/index.php/Служебная:Моё_обсуждение
/index.php/Служебная:Мой_вклад
/index.php?title=Служебная:Создать_учётную_запись&returnto=Категория:Растения+–+Цветы
/index.php?title=Служебная:Вход&returnto=Категория:Растения+–+Цветы
/index.php/Категория:Растения_–_Цветы
/index.php?title=Обсуждение_категории:Растения_–_Цветы&action=edit&redlink=1
/index.php/Категория:Растения_–_Цветы
/index.php?title=Категория:Растения_–_Цветы&action=edit
/index.php?title=Категория:Растения_–_Цветы&action=history
/index.php/Краткая_энциклопедия

/index.php/Набедренная_повязка
/index.php/Набедренник
/index.php/Наголовье
/index.php/Награды
/index.php/Наплечник
/index.php/Облачение
/index.php/Обувь
/index.php/Одежда
/index.php/Ожерелье
/index.php/Опахало
/index.php/Паллий
/index.php/Передник
/index.php/Перстень
/index.php/Перчатки
/index.php/Петас
/index.php/Печать
/index.php/Плат
/index.php/Платок_головной
/index.php/Платок_носовой
/index.php/Плащ
/index.php/Плюмаж
/index.php/Покров,_плат
/index.php/Покрывало
/index.php/Полог
/index.php/Пояс_Шесемтет
/index.php/Пояс,_деталь_одежды
/index.php/Пряжка
/index.php/Пуговицы
/index.php/Религиозные_одежды
/index.php/Риза
/index.php/Рокетта
/index.php/Ряса
/index.php/Саван
/index.php/Саккос
/index.php/Скипетр
/index.php/Стихарь
/index.php/Стола
/index.php/Сутана
/index.php/Тиара
/index.php/Тирс
/index.php/Торквес
/index.php/Трон
/index.php/Туника
/index.php/Туфли_домашние
/index.php/Тюрбан
/index.php/Украшение
/index.php/Урим_и_Тумим
/index.php/Фартук
/index.php/Фасции
/index.php/Флаг
/i

/index.php/Гексаграммы_И-цзин
/index.php/Геометрические_символы
/index.php/Горизонтальная_линия
/index.php/Графические_символы
/index.php/Гротеск
/index.php/Диск
/index.php/Дуга
/index.php/Египетский_крест
/index.php/Замковый_камень
/index.php/Звезда_восьмиконечная
/index.php/Звезда_девятиконечная
/index.php/Звезда_морей
/index.php/Звезда_пятиконечная
/index.php/Звезда_семиконечная
/index.php/Звезда_четырёхконечная
/index.php/Звезда_шестиконечная
/index.php/Зигзаг
/index.php/Знаки_Хекер
/index.php/Изгиб
/index.php/Изображение
/index.php/Искажённые_формы
/index.php/Кадукей
/index.php/Квадрат
/index.php/Квинкунс
/index.php/Ключ_Мельхиседека
/index.php/Конус
/index.php/Крест
/index.php/Крест_вилообразный
/index.php/Крест_голгофский
/index.php/Крест_греческий
/index.php/Крест_кельтский
/index.php/Крест_косой
/index.php/Крест_костыльный
/index.php/Крест_латинский
/index.php/Крест_лилиевидный
/index.php/Крест_мальтийский
/index.php/Крест_огненный
/index.php/Крест_патриарший
/index.php/Крест_

/index.php/Батрадз
/index.php/Баубо
/index.php/Бахрам_Гур
/index.php/Бел,_сын_Посейдона
/index.php/Беллерофонт
/index.php/Беовульф
/index.php/Биант
/index.php/Библида
/index.php/Бизант
/index.php/Бодончар
/index.php/Бома
/index.php/Бората
/index.php/Бореады
/index.php/Борте-Чино
/index.php/Боткий_Ширтка
/index.php/Боян
/index.php/Бран
/index.php/Бранх
/index.php/Брисеида
/index.php/Бузиг
/index.php/Бунг
/index.php/Бусирис
/index.php/Бут,_аргонавт
/index.php/Бут,_сын_Борея
/index.php/Бут,_сын_Пандиона
/index.php/Буха-нойон_бабай
/index.php/Бхарата
/index.php/Бхима
/index.php/Бхишма
/index.php/Бхуджью
/index.php/Ваджранга
/index.php/Вайделоты
/index.php/Вако-Нана
/index.php/Вальмики
/index.php/Ван_Гон
/index.php/Ван_Хай
/index.php/Ванцзы_Цяо
/index.php/Васитта
/index.php/Васудева
/index.php/Вена
/index.php/Видарафш
/index.php/Видевут_и_Брутен
/index.php/Вирбий
/index.php/Виргиния
/index.php/Вишвамитра
/index.php/Виштаспа
/index.php/Во-цюань
/index.php/Волхв
/index.php/Восемь_бессмертных


/index.php/Архетип
/index.php/Атрибуты
/index.php/Басня
/index.php/Близнечные_мифы
/index.php/Богиня-мать
/index.php/Божественный_посланник
/index.php/Вестники_богов
/index.php/Время_мифическое
/index.php/Геометрические_символы
/index.php/Двуполые_существа
/index.php/Демиург
/index.php/Древо_Жизни
/index.php/Древо_мировое
/index.php/Древо_познания
/index.php/Дуалистические_мифы
/index.php/Дуалистические_символы
/index.php/Дух,_духи
/index.php/Живопись
/index.php/Загадка
/index.php/Заговоры
/index.php/Запах
/index.php/Знак
/index.php/Идиллия
/index.php/Иероглифика
/index.php/Изобразительное_искусство_и_мифология
/index.php/Инверсия
/index.php/Индивидуация
/index.php/История_и_мифы
/index.php/Катастрофа
/index.php/Конец_Света
/index.php/Конфуцианство
/index.php/Космическое_яйцо
/index.php/Космогонические_мифы
/index.php/Космогония
/index.php/Космос
/index.php/Культовые_мифы
/index.php/Культурный_герой
/index.php/Легенды_и_мифы
/index.php/Литература_и_мифы
/index.php/Логос
/index.php/Луна

/index.php/Абрикосовое_дерево_(геральд.)
/index.php/Авроры,_цвет_(геральд.)
/index.php/Агнец_пасхальный_(геральд.)
/index.php/Администратор_(геральд.)
/index.php/Азур_(геральд.)
/index.php/Аист_(геральд.)
/index.php/Акант_(геральд.)
/index.php/Акация_(геральд.)
/index.php/Акула_(геральд.)
/index.php/Албанская_шапка_(геральд.)
/index.php/Алерион_(геральд.)
/index.php/Алкион_(геральд.)
/index.php/Аллюр_(геральд.)
/index.php/Алмаз_(геральд.)
/index.php/Альпака_(геральд.)
/index.php/Амфиптера_(геральд.)
/index.php/Амфисбена_(геральд.)
/index.php/Ангел_(геральд.)
/index.php/Английский_щит_(геральд.)
/index.php/Андреевский_(геральд.)
/index.php/Андреевский_крест_(геральд.)
/index.php/Андреевский_флаг_(геральд.)
/index.php/Анкер_(геральд.)
/index.php/Анкершток_(геральд.)
/index.php/Антилопа_(геральд.)
/index.php/Античные_боги_и_персонажи_(геральд.)
/index.php/Антониевский_крест_(геральд.)
/index.php/Апельсин_(геральд.)
/index.php/Апостол_(геральд.)
/index.php/Арабеска_(геральд.)
/index.php/Ар

/index.php/Служебная:Спецстраницы
/index.php?title=Эльзен&printable=yes
/index.php?title=Эльзен&oldid=154763
/index.php?title=Эльзен&action=info
/index.php/Энциклопедия_знаков_и_символов:Политика_конфиденциальности
/index.php/Энциклопедия_знаков_и_символов:Описание
/index.php/Энциклопедия_знаков_и_символов:Отказ_от_ответственности
http://www.symbolarium.ru/index.php?title=Эльзен&mobileaction=toggle_view_mobile
//www.mediawiki.org/
//www.mediawiki.org/wiki/Special:MyLanguage/Help:What_links_here
/index.php/Символика
#mw-head
#p-search
/index.php?title=Служебная:Ссылки_сюда/Символика&hidetrans=1
/index.php?title=Служебная:Ссылки_сюда/Символика&hidelinks=1
/index.php?title=Служебная:Ссылки_сюда/Символика&hideredirs=1
/index.php/Символика
/index.php?title=Служебная:Ссылки_сюда/Символика&limit=20
/index.php?title=Служебная:Ссылки_сюда/Символика&limit=50
/index.php?title=Служебная:Ссылки_сюда/Символика&limit=100
/index.php?title=Служебная:Ссылки_сюда/Символика&limit=250
/index.php?title=Служ

# Попытка использовать методы из 
## http://rusvectores.org/ru/models/
## https://github.com/akutuzov/webvectors/blob/master/preprocessing/rusvectores_tutorial.ipynb
## https://github.com/akutuzov/webvectors/blob/master/preprocessing/rus_preprocessing_udpipe.py

In [12]:
from __future__ import print_function
from __future__ import division
from future import standard_library
import sys
import os
import wget
import re
from ufal.udpipe import Model, Pipeline




def num_replace(word):
    newtoken = 'x' * len(word)
    return newtoken


def clean_token(token, misc):
    """
    :param token:  токен (строка)
    :param misc:  содержимое поля "MISC" в CONLLU (строка)
    :return: очищенный токен (строка)
    """
    out_token = token.strip().replace(' ', '')
    if token == 'Файл' and 'SpaceAfter=No' in misc:
        return None
    return out_token


def clean_lemma(lemma, pos):
    """
    :param lemma: лемма (строка)
    :param pos: часть речи (строка)
    :return: очищенная лемма (строка)
    """
    out_lemma = lemma.strip().replace(' ', '').replace('_', '').lower()
    if '|' in out_lemma or out_lemma.endswith('.jpg') or out_lemma.endswith('.png'):
        return None
    if pos != 'PUNCT':
        if out_lemma.startswith('«') or out_lemma.startswith('»'):
            out_lemma = ''.join(out_lemma[1:])
        if out_lemma.endswith('«') or out_lemma.endswith('»'):
            out_lemma = ''.join(out_lemma[:-1])
        if out_lemma.endswith('!') or out_lemma.endswith('?') or out_lemma.endswith(',') \
                or out_lemma.endswith('.'):
            out_lemma = ''.join(out_lemma[:-1])
    return out_lemma


def list_replace(search, replacement, text):
    search = [el for el in search if el in text]
    for c in search:
        text = text.replace(c, replacement)
    return text


def unify_sym(text):  # принимает строку в юникоде
    text = list_replace \
        ('\u00AB\u00BB\u2039\u203A\u201E\u201A\u201C\u201F\u2018\u201B\u201D\u2019', '\u0022', text)

    text = list_replace \
        ('\u2012\u2013\u2014\u2015\u203E\u0305\u00AF', '\u2003\u002D\u002D\u2003', text)

    text = list_replace('\u2010\u2011', '\u002D', text)

    text = list_replace \
            (
            '\u2000\u2001\u2002\u2004\u2005\u2006\u2007\u2008\u2009\u200A\u200B\u202F\u205F\u2060\u3000',
            '\u2002', text)

    text = re.sub('\u2003\u2003', '\u2003', text)
    text = re.sub('\t\t', '\t', text)

    text = list_replace \
            (
            '\u02CC\u0307\u0323\u2022\u2023\u2043\u204C\u204D\u2219\u25E6\u00B7\u00D7\u22C5\u2219\u2062',
            '.', text)

    text = list_replace('\u2217', '\u002A', text)

    text = list_replace('…', '...', text)

    text = list_replace('\u2241\u224B\u2E2F\u0483', '\u223D', text)

    text = list_replace('\u00C4', 'A', text)  # латинская
    text = list_replace('\u00E4', 'a', text)
    text = list_replace('\u00CB', 'E', text)
    text = list_replace('\u00EB', 'e', text)
    text = list_replace('\u1E26', 'H', text)
    text = list_replace('\u1E27', 'h', text)
    text = list_replace('\u00CF', 'I', text)
    text = list_replace('\u00EF', 'i', text)
    text = list_replace('\u00D6', 'O', text)
    text = list_replace('\u00F6', 'o', text)
    text = list_replace('\u00DC', 'U', text)
    text = list_replace('\u00FC', 'u', text)
    text = list_replace('\u0178', 'Y', text)
    text = list_replace('\u00FF', 'y', text)
    text = list_replace('\u00DF', 's', text)
    text = list_replace('\u1E9E', 'S', text)

    currencies = list \
            (
            '\u20BD\u0024\u00A3\u20A4\u20AC\u20AA\u2133\u20BE\u00A2\u058F\u0BF9\u20BC\u20A1\u20A0\u20B4\u20A7\u20B0\u20BF\u20A3\u060B\u0E3F\u20A9\u20B4\u20B2\u0192\u20AB\u00A5\u20AD\u20A1\u20BA\u20A6\u20B1\uFDFC\u17DB\u20B9\u20A8\u20B5\u09F3\u20B8\u20AE\u0192'
        )

    alphabet = list \
            (
            '\t\n\r абвгдеёзжийклмнопрстуфхцчшщьыъэюяАБВГДЕЁЗЖИЙКЛМНОПРСТУФХЦЧШЩЬЫЪЭЮЯ,.[]{}()=+-−*&^%$#@!~;:0123456789§/\|"abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ ')

    alphabet.append("'")

    allowed = set(currencies + alphabet)

    cleaned_text = [sym for sym in text if sym in allowed]
    cleaned_text = ''.join(cleaned_text)

    return cleaned_text





UDPipe model not found. Downloading...

Loading the model...
Processing input...


In [14]:
import wget

udpipe_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
text_url = 'https://rusvectores.org/static/henry_sobolya.txt'

modelfile = wget.download(udpipe_url)
textfile = wget.download(text_url)

In [15]:
def process(pipeline, text='Строка', keep_pos=True, keep_punct=False):
    entities = {'PROPN'}
    named = False
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [l for l in processed.split('\n') if not l.startswith('#')]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split('\t') for w in content if w]

    for t in tagged:
        if len(t) != 10:
            continue
        (word_id, token, lemma, pos, xpos, feats, head, deprel, deps, misc) = t
        token = clean_token(token, misc)
        lemma = clean_lemma(lemma, pos)
        if not lemma or not token:
            continue
        if pos in entities:
            if '|' not in feats:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            morph = {el.split('=')[0]: el.split('=')[1] for el in feats.split('|')}
            if 'Case' not in morph or 'Number' not in morph:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            if not named:
                named = True
                mem_case = morph['Case']
                mem_number = morph['Number']
            if morph['Case'] == mem_case and morph['Number'] == mem_number:
                memory.append(lemma)
                if 'SpacesAfter=\\n' in misc or 'SpacesAfter=\s\\n' in misc:
                    named = False
                    past_lemma = '::'.join(memory)
                    memory = []
                    tagged_propn.append(past_lemma + '_PROPN ')
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))
        else:
            if not named:
                if pos == 'NUM' and token.isdigit():  # Заменяем числа на xxxxx той же длины
                    lemma = num_replace(token)
                tagged_propn.append('%s_%s' % (lemma, pos))
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))

    if not keep_punct:
        tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']
    if not keep_pos:
        tagged_propn = [word.split('_')[0] for word in tagged_propn]
    return tagged_propn

In [16]:
from ufal.udpipe import Model, Pipeline
import os
import re

def tag_ud(text='Текст нужно передать функции в виде строки!', modelfile='udpipe_syntagrus.model'):
    udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
    udpipe_filename = udpipe_model_url.split('/')[-1]

    if not os.path.isfile(modelfile):
        print('UDPipe model not found. Downloading...', file=sys.stderr)
        wget.download(udpipe_model_url)

    print('\nLoading the model...', file=sys.stderr)
    model = Model.load(modelfile)
    process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

    print('Processing input...', file=sys.stderr)
    for line in text:
        # line = unify_sym(line.strip()) # здесь могла бы быть ваша функция очистки текста
        output = process(process_pipeline, text=line)
        print(' '.join(output))

In [23]:
#text = open(textfile, 'r', encoding='utf-8').read()
ascii_letters = '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'
#text = open('Test\Ящик.txt', 'r', encoding='utf-8').read()
text = open('Lyrics\Незнакомка.txt', 'r', encoding='utf-8').read()
for char in ascii_letters:
    text=text.replace(char,'')
tag_ud(text=text, modelfile=modelfile)


Loading the model...
Processing input...


по_NOUN
о_ADP
в_ADP
быть_NOUN
число_NOUN
быть_NOUN
река_NOUN
а_CCONJ
метр_NOUN
называть_NOUN
а_CCONJ
дорога_NOUN
река_NOUN
быть_NOUN
с_ADP
том_NOUN
о_ADP
река_NOUN
а_CCONJ
называть_NOUN
а_CCONJ
метр_NOUN
и_CCONJ
г_PROPN 
о_ADP
река_NOUN
я_PRON
число_NOUN
и_CCONJ
и_NOUN
в_ADP
о_ADP
с_NOUN
дорога_NOUN
в_ADP
х_CCONJ
дорога_NOUN
и_CCONJ
к_ADP
и_CCONJ
господин_NOUN
лист_NOUN
в_ADP
х_CCONJ
и_CCONJ
подобный_NOUN
река_NOUN
а_CCONJ
в_ADP
и_CCONJ
том_NOUN
о_ADP
к_ADP
река_NOUN
и_CCONJ
к_ADP
а_CCONJ
метр_NOUN
и_CCONJ
подобный_NOUN
ь_NOUN
я_PRON
называть_NOUN
ы_NOUN
метр_NOUN
и_CCONJ
в_ADP
быть_NOUN
с_ADP
быть_NOUN
называть_NOUN
называть_NOUN
и_CCONJ
и_NOUN
и_CCONJ
том_NOUN
лист_NOUN
быть_NOUN
том_NOUN
в_ADP
о_ADP
река_NOUN
называть_NOUN
ы_NOUN
и_NOUN
дорога_NOUN
в_ADP
х_CCONJ
в_ADP
дорога_NOUN
а_CCONJ
лист_NOUN
и_CCONJ
называть_NOUN
а_CCONJ
дорога_NOUN
подобный_NOUN
ы_NOUN
лист_NOUN
ь_NOUN
ю_VERB
подобный_NOUN
быть_NOUN
река_NOUN
быть_NOUN
в_ADP
лист_NOUN
о_ADP
число_NOUN
называть_NOUN
о_ADP
и_NO

# Немного работы с частотами

In [105]:
from docx import Document
from docx.shared import RGBColor
from docx.shared import Pt
from docx.enum.style import WD_STYLE_TYPE

document_hand = Document('Docs\Рука.docx')
document_moon = Document('Docs\Луна.docx')
document_home = Document('Docs\Дом.docx')




In [258]:
def getText(filename):
    doc = Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

In [173]:
from docx import Document
from docx.shared import RGBColor
from docx.shared import Pt
from docx.enum.style import WD_STYLE_TYPE

temp_color_emotion_dict={}
emotion_dict={7:'Symbol',6:'Anger',4:'Happy',16:'Fear',5:'Sadness',9:'Suprise_Bad',3:'Suprise_Good'}

def emotion_symbol_amount(word):
    with open(word+'_частота.txt','w',encoding='utf-8') as f:
        f.write('Слово \"{}\" встречается как соответствующая эмоция или символ следующим образом:'.format(word))
        for emote in temp_color_emotion_dict:
            f.write(str(emotion_dict[emote]) + ' '+str(temp_color_emotion_dict[emote])+'\n')
    
def emotion_freq(word):
    emote_sum=0
    for emote in temp_color_emotion_dict:
        if(emote!=7):
            emote_sum+=temp_color_emotion_dict[emote]
    with open(word+'_частота.txt','w',encoding='utf-8') as f:
        f.write('Слово \"{}\" встречается как соответствующая эмоция:\n'.format(word))
        for emote in temp_color_emotion_dict:
            f.write(str(emotion_dict[emote]) +' '+ str(temp_color_emotion_dict[emote]/emote_sum)+'\n')
    
def Read_Docs_Emotion(filename):
    temp_color_emotion_dict={}
    document_hand = Document('Docs\\'+filename+'.docx')
    for para in document_hand.paragraphs:
        for run in para.runs:
            if(run.font.highlight_color is not None):
                if (run.font.highlight_color in temp_color_emotion_dict):
                    temp_color_emotion_dict[run.font.highlight_color]+=1
                else:
                    temp_color_emotion_dict[run.font.highlight_color]=1
    emotion_symbol_amount(filename)
    emotion_freq(filename)

    
Read_Docs_Emotion('Рука')
Read_Docs_Emotion('Луна')
Read_Docs_Emotion('Дом') 

"\ntemp_color_emotion_dict={}\ndocument_hand = Document('Docs\\Человек.docx')\nfor para in document_hand.paragraphs:\n    for run in para.runs:\n        if(run.font.highlight_color is not None):\n            if (run.font.highlight_color in temp_color_emotion_dict):\n                temp_color_emotion_dict[run.font.highlight_color]+=1\n            else:\n                temp_color_emotion_dict[run.font.highlight_color]=1\n\nemotion_symbol_amount('Человек')\n    \nemotion_freq('Человек')   \n"

In [231]:
print(len(TO_DELETE))
for token in TO_DELETE:
    print(token)

68858
﻿часть
один
часть
сила
вечно
хотеть
вечно
совершать
благо
гете
фауст
глава
никогда
разговаривать
неизвестный
однажды
весною
небывалый
жаркое
закат
москва
патриарший
пруд
появиться
гражданин
один
одетый
летний
серенький
пара
маленький
рост
упитанный
приличный
шляпа
пирожок
рука
выбрить
лицо
помещаться
сверхъестественный
размер
очки
чёрный
рогов
оправа
два
плечистый
рыжеватый
вихрастый
молодая
человек
заломить
затылок
клетчатый
кепка
ковбойка
жёваный
белых
брюки
чёрный
тапочки
один
иной
михаил
александр
берлиоз
председатель
правление
один
крупный
московский
литературный
ассоциация
сокращённо
именовать
массолит
редактор
толстой
художественный
журнал
молодая
спутник
поэт
иван
николай
понырёв
пишущий
псевдоним
бездомный
попасть
тень
зеленеть
писатель
один
долг
броситься
пестро
раскрасить
будочка
надпись
пиво
вода
следовать
отметить
один
странность
страшный
майский
вечер
будочка
весь
аллея
параллельный
малый
бронная
улица
оказаться
один
человек
кажется
дышать
солнце
раскалить
москва
су

пояснить
государственный
библиотека
обнаружить
подлинный
рукопись
чернокнижник
герберт
аврилакский
десять
век
требоваться
разобрать
единственный
мир
специалист
историк
больший
облегчение
уважение
спросить
берлиоз
историк
подтвердить
учёный
добавить
село
город
сегодня
вечером
патриарший
пруд
интересный
история
крайне
удивиться
редактор
поэт
профессор
поманить
оба
наклониться
он
прошептать
иметь
вид
иисус
существовать
видеть
профессор
принужденно
улыбнуться
отозваться
берлиоз
уважаемый
ваш
большой
знание
сам
это
вопрос
придерживаться
точка
зрение
никакой
точка
зрение
ответить
странный
профессор
просто
существовать
требоваться
какой-нибудь
доказательство
начать
берлиоз
доказательство
никакой
требоваться
ответить
профессор
заговорить
негромко
причём
акцент
почему-то
пропасть
просто
бел
плащ
глава
понтия
пилат
бел
плащ
кровавый
подбой
шаркать
кавалерийский
походка
ранний
утром
четырнадцать
число
весенний
месяц
нисан
крытый
колоннада
два
крыло
дворец
ирод
великое
выйти
прокуратор
иудей
понти

матвей
донестись
высокий
мучаять
голос
всё-таки
говорить
храм
толпа
базар
голос
отвечать
казалось
колоть
пилат
висок
невыразимый
мучительный
голос
говорить
игемона
говорить
рухнуть
храм
старый
вера
создаться
новый
храм
истина
сказать
понятный
зачем
бродяга
базар
смущать
народ
рассказывать
истина
который
иметь
представление
такой
истина
прокуратор
подумать
бог
спрашивать
что-то
ненужный
суд
служить
померещиться
чаша
тёмный
жидкость
вновь
услышать
голос
истина
прежде
болеть
голова
болеть
сильно
малодушно
помышлять
смерть
сила
говорить
я
ты
трудно
глядеть
невольно
являться
твой
палач
огорчать
мочь
думать
что-нибудь
мечтать
пришлый
твой
собака
единственный
по-видимому
существо
который
привязать
мучение
твой
кончиться
голова
пройти
секретарь
вытаращить
глаз
арестант
дописать
слово
пилат
поднять
мученический
глаз
арестант
увидеть
солнце
довольно
высоко
стоить
гипподром
пробраться
колоннада
подползать
стоптать
сандалия
иешуа
сторониться
солнце
прокуратор
подняться
кресло
сжать
голова
рука
жел

погаснуть
какой-то
короткий
другой
мысль
бессмертие
пришлый
бессмертие
бессмертие
пришлый
это
понять
прокуратор
мысль
загадочный
бессмертие
заставить
похолодеть
солнцепёк
хорошо
сказать
пилат
оглянуться
окинуть
взор
видимый
удивиться
произойти
перемена
пропасть
отяготить
роза
куст
пропасть
кипарис
окаймлять
верхний
терраса
гранатовый
дерево
белый
статуя
зелень
сам
зелень
поплыть
вместо
какой-то
багровый
гуща
закачаться
водоросль
двинуться
куда-то
вместе
они
двинуться
пилат
теперь
уносить
удушать
обжигать
самый
страшный
гнев
гнев
бессилие
тесно
вымолвить
пилат
тесно
холодный
влажный
рука
рвануть
пряжка
ворот
плащ
упасть
песок
сегодня
душный
где-то
идти
гроза
отозваться
каифа
сводить
глаз
покраснеть
лицо
прокуратор
предвидеть
мука
который
предстоять
страшный
месяц
нисан
год
сказать
пилат
оттого
душный
тесно
стать
ты
каифа
сузить
глаз
пилат
улыбнуться
добавить
поберечь
первосвященник
тёмный
глаз
первосвященник
блеснуть
худой
ранее
прокуратор
выразить
свой
лицо
удивление
слышать
прокуратор

KeyboardInterrupt: 

In [246]:
input_file = csv.DictReader(open("Docs\\bigsize_with_user_tags.csv"))
some_count_adsadsad=0
for row in input_file:
    some_count_adsadsad+=1
print(some_count_adsadsad)
some_count_adsadsad=0
for token in TO_DELETE:
    some_count_adsadsad+=1
print(some_count_adsadsad)

14189
2050


In [250]:
import csv
count=0

emotion_arr=[0,0,0,0,0,0,0,0,0,0]
emotion_name=['Positive','Negative','Anger','Anticipation','Disgust','Fear','Joy','Sadness','Surprise','Trust']
input_file = csv.DictReader(open("Docs\\bigsize_with_user_tags.csv"))

max_limit=100
for row in input_file:
    inc=0
    print(row['num'])
    for token in TO_DELETE:
        inc+=1
        if inc>100:
            break
        another=prepare_text_for_lda(row['word'])
        if (len(another)==0):
            continue
        if (token==another[0]):
            for i in range(10):
                emotion_arr[i]+=int(row[emotion_name[i]])                   
print(emotion_arr)
with open('emotion_freq.txt','w',encoding='utf-8') as f:
    for i in range(10):
        f.write(emotion_name[i] + ' '+str(emotion_arr[i])+'\n')



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

3498
3499
3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697


6774
6775
6776
6777
6778
6779
6780
6781
6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973


10042
10043
10044
10045
10046
10047
10048
10049
10050
10051
10052
10053
10054
10055
10056
10057
10058
10059
10060
10061
10062
10063
10064
10065
10066
10067
10068
10069
10070
10071
10072
10073
10074
10075
10076
10077
10078
10079
10080
10081
10082
10083
10084
10085
10086
10087
10088
10089
10090
10091
10092
10093
10094
10095
10096
10097
10098
10099
10100
10101
10102
10103
10104
10105
10106
10107
10108
10109
10110
10111
10112
10113
10114
10115
10116
10117
10118
10119
10120
10121
10122
10123
10124
10125
10126
10127
10128
10129
10130
10131
10132
10133
10134
10135
10136
10137
10138
10139
10140
10141
10142
10143
10144
10145
10146
10147
10148
10149
10150
10151
10152
10153
10154
10155
10156
10157
10158
10159
10160
10161
10162
10163
10164
10165
10166
10167
10168
10169
10170
10171
10172
10173
10174
10175
10176
10177
10178
10179
10180
10181
10182
10183
10184
10185
10186
10187
10188
10189
10190
10191
10192
10193
10194
10195
10196
10197
10198
10199
10200
10201
10202
10203
10204
10205
10206
10207
1020

12772
12773
12774
12775
12776
12777
12778
12779
12780
12781
12782
12783
12784
12785
12786
12787
12788
12789
12790
12791
12792
12793
12794
12795
12796
12797
12798
12799
12800
12801
12802
12803
12804
12805
12806
12807
12808
12809
12810
12811
12812
12813
12814
12815
12816
12817
12818
12819
12820
12821
12822
12823
12824
12825
12826
12827
12828
12829
12830
12831
12832
12833
12834
12835
12836
12837
12838
12839
12840
12841
12842
12843
12844
12845
12846
12847
12848
12849
12850
12851
12852
12853
12854
12855
12856
12857
12858
12859
12860
12861
12862
12863
12864
12865
12866
12867
12868
12869
12870
12871
12872
12873
12874
12875
12876
12877
12878
12879
12880
12881
12882
12883
12884
12885
12886
12887
12888
12889
12890
12891
12892
12893
12894
12895
12896
12897
12898
12899
12900
12901
12902
12903
12904
12905
12906
12907
12908
12909
12910
12911
12912
12913
12914
12915
12916
12917
12918
12919
12920
12921
12922
12923
12924
12925
12926
12927
12928
12929
12930
12931
12932
12933
12934
12935
12936
12937
1293

In [1]:
#Поиск всех ссылок в ссылках
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
from urllib.parse import unquote
from urllib.error import URLError, HTTPError
import re

def get_start_page(target):
    return 'http://sensefreq.ruslang.ru/w/{}'.format(quote(target))
def get_page_url(target):
    return "http://sensefreq.ruslang.ru{}".format(quote(target))

print('start')
links = []
html_page = urllib.request.urlopen(get_start_page(unquote('RNC%20%28adjectives%29')))
soup = BeautifulSoup(html_page)
for link in soup.find_all('a', href=True):
    links.append(link.get('href'))

html_page = urllib.request.urlopen(get_start_page(unquote('RNC%20%28nouns%29')))
soup = BeautifulSoup(html_page)
for link in soup.find_all('a', href=True):
    links.append(link.get('href'))
    
html_page = urllib.request.urlopen(get_start_page(unquote('RNC%20%28adjectives%29')))
soup = BeautifulSoup(html_page)
for link in soup.find_all('a', href=True):
    links.append(link.get('href'))

symbol_links=[]
num_links=len(links)
count=0
for link in links:
    count+=1
    print('Это исходный ', unquote(link))
    print('{} из {}'.format(count,num_links))
    try:
        SymbolFile = urllib.request.urlopen(get_page_url(unquote(link)))       
    except URLError as e:
        continue
    else:
        SymbolHtml = SymbolFile.read()
        SymbolFile.close()
        soup = BeautifulSoup(SymbolHtml)
        for link in soup.find_all('a', href=True):
            symbol_links.append(link.get('href'))

print(symbol_links)
print('Ready')      



start
Это исходный  /w/RNC (adjectives)?homonyms=yes
1 из 957
Это исходный  /w/RNC (adjectives)?homonyms=no
2 из 957
Это исходный  /w/RNC (adjectives)/абрикосовый
3 из 957
Это исходный  /w/RNC (adjectives)/абсолютный
4 из 957
Это исходный  /w/RNC (adjectives)/абстрактный
5 из 957
Это исходный  /w/RNC (adjectives)/автономный
6 из 957
Это исходный  /w/RNC (adjectives)/авторитетный
7 из 957
Это исходный  /w/RNC (adjectives)/аграрный
8 из 957
Это исходный  /w/RNC (adjectives)/агрессивный
9 из 957
Это исходный  /w/RNC (adjectives)/адекватный
10 из 957
Это исходный  /w/RNC (adjectives)/административный
11 из 957
Это исходный  /w/RNC (adjectives)/адский
12 из 957
Это исходный  /w/RNC (adjectives)/азиатский
13 из 957
Это исходный  /w/RNC (adjectives)/активный
14 из 957
Это исходный  /w/RNC (adjectives)/акционерный
15 из 957
Это исходный  /w/RNC (adjectives)/алкогольный
16 из 957
Это исходный  /w/RNC (adjectives)/алмазный
17 из 957
Это исходный  /w/RNC (adjectives)/алчный
18 из 957
Это исходный

Это исходный  /w/RNC (adjectives)/верхний
150 из 957
Это исходный  /w/RNC (adjectives)/весь
151 из 957
Это исходный  /w/RNC (adjectives)/весёлый
152 из 957
Это исходный  /w/RNC (adjectives)/ветреный
153 из 957
Это исходный  /w/RNC (adjectives)/ветхий
154 из 957
Это исходный  /w/RNC (adjectives)/вечный
155 из 957
Это исходный  /w/RNC (adjectives)/взволнованный
156 из 957
Это исходный  /w/RNC (adjectives)/вздорный
157 из 957
Это исходный  /w/RNC (adjectives)/взрослый
158 из 957
Это исходный  /w/RNC (adjectives)/видимый
159 из 957
Это исходный  /w/RNC (adjectives)/видный
160 из 957
Это исходный  /w/RNC (adjectives)/виноватый
161 из 957
Это исходный  /w/RNC (adjectives)/виртуозный
162 из 957
Это исходный  /w/RNC (adjectives)/вишнёвый
163 из 957
Это исходный  /w/RNC (adjectives)/влажный
164 из 957
Это исходный  /w/RNC (adjectives)/властный
165 из 957
Это исходный  /w/RNC (adjectives)/влюблённый
166 из 957
Это исходный  /w/RNC (adjectives)/внешний
167 из 957
Это исходный  /w/RNC (adjectives)

Это исходный  /w/RNC (nouns)/арест
307 из 957
Это исходный  /w/RNC (nouns)/аристократия
308 из 957
Это исходный  /w/RNC (nouns)/арка
309 из 957
Это исходный  /w/RNC (nouns)/армия
310 из 957
Это исходный  /w/RNC (nouns)/аромат
311 из 957
Это исходный  /w/RNC (nouns)/арсенал
312 из 957
Это исходный  /w/RNC (nouns)/артерия
313 из 957
Это исходный  /w/RNC (nouns)/артиллерия
314 из 957
Это исходный  /w/RNC (nouns)/артист
315 из 957
Это исходный  /w/RNC (nouns)/архипелаг
316 из 957
Это исходный  /w/RNC (nouns)/архитектор
317 из 957
Это исходный  /w/RNC (nouns)/архитектура
318 из 957
Это исходный  /w/RNC (nouns)/ассистент
319 из 957
Это исходный  /w/RNC (nouns)/ассоциация
320 из 957
Это исходный  /w/RNC (nouns)/астрономия
321 из 957
Это исходный  /w/RNC (nouns)/асфальт
322 из 957
Это исходный  /w/RNC (nouns)/атака
323 из 957
Это исходный  /w/RNC (nouns)/ателье
324 из 957
Это исходный  /w/RNC (nouns)/атмосфера
325 из 957
Это исходный  /w/RNC (nouns)/атом
326 из 957
Это исходный  /w/RNC (nouns)

Это исходный  /w/RNC (nouns)/ведро
481 из 957
Это исходный  /w/RNC (nouns)/ведьма
482 из 957
Это исходный  /w/RNC (nouns)/веер
483 из 957
Это исходный  /w/RNC (nouns)/век
484 из 957
Это исходный  /w/RNC (nouns)/великан
485 из 957
Это исходный  /w/RNC (nouns)/величина
486 из 957
Это исходный  /w/RNC (nouns)/венец
487 из 957
Это исходный  /w/RNC (nouns)/вера
488 из 957
Это исходный  /w/RNC (nouns)/вермишель
489 из 957
Это исходный  /w/RNC (nouns)/верность
490 из 957
Это исходный  /w/RNC (nouns)/вероятность
491 из 957
Это исходный  /w/RNC (nouns)/верх
492 из 957
Это исходный  /w/RNC (nouns)/верхушка
493 из 957
Это исходный  /w/RNC (nouns)/вершина
494 из 957
Это исходный  /w/RNC (nouns)/вес
495 из 957
Это исходный  /w/RNC (nouns)/весы
496 из 957
Это исходный  /w/RNC (nouns)/ветер
497 из 957
Это исходный  /w/RNC (nouns)/ветка
498 из 957
Это исходный  /w/RNC (nouns)/веха
499 из 957
Это исходный  /w/RNC (nouns)/вечер
500 из 957
Это исходный  /w/RNC (nouns)/вечность
501 из 957
Это исходный  /w

Это исходный  /w/RNC (nouns)/город
654 из 957
Это исходный  /w/RNC (nouns)/городок
655 из 957
Это исходный  /w/RNC (nouns)/горох
656 из 957
Это исходный  /w/RNC (nouns)/горсть
657 из 957
Это исходный  /w/RNC (nouns)/горчица
658 из 957
Это исходный  /w/RNC (nouns)/горшок
659 из 957
Это исходный  /w/RNC (nouns)/горючее
660 из 957
Это исходный  /w/RNC (nouns)/горячка
661 из 957
Это исходный  /w/RNC (nouns)/господин
662 из 957
Это исходный  /w/RNC (nouns)/господство
663 из 957
Это исходный  /w/RNC (nouns)/гостиная
664 из 957
Это исходный  /w/RNC (nouns)/гость
665 из 957
Это исходный  /w/RNC (nouns)/государство
666 из 957
Это исходный  /w/RNC (nouns)/грабли
667 из 957
Это исходный  /w/RNC (nouns)/гравюра
668 из 957
Это исходный  /w/RNC (nouns)/градус
669 из 957
Это исходный  /w/RNC (nouns)/гражданин
670 из 957
Это исходный  /w/RNC (nouns)/грамм
671 из 957
Это исходный  /w/RNC (nouns)/грамота
672 из 957
Это исходный  /w/RNC (nouns)/грань
673 из 957
Это исходный  /w/RNC (nouns)/графа
674 из 9

Это исходный  /w/RNC (adjectives)/бодрый
807 из 957
Это исходный  /w/RNC (adjectives)/боевой
808 из 957
Это исходный  /w/RNC (adjectives)/божественный
809 из 957
Это исходный  /w/RNC (adjectives)/божий
810 из 957
Это исходный  /w/RNC (adjectives)/бойкий
811 из 957
Это исходный  /w/RNC (adjectives)/боковой
812 из 957
Это исходный  /w/RNC (adjectives)/болгарский
813 из 957
Это исходный  /w/RNC (adjectives)/более
814 из 957
Это исходный  /w/RNC (adjectives)/болезненный
815 из 957
Это исходный  /w/RNC (adjectives)/больной
816 из 957
Это исходный  /w/RNC (adjectives)/больше
817 из 957
Это исходный  /w/RNC (adjectives)/больший
818 из 957
Это исходный  /w/RNC (adjectives)/большой
819 из 957
Это исходный  /w/RNC (adjectives)/боязливый
820 из 957
Это исходный  /w/RNC (adjectives)/братский
821 из 957
Это исходный  /w/RNC (adjectives)/брачный
822 из 957
Это исходный  /w/RNC (adjectives)/брезгливый
823 из 957
Это исходный  /w/RNC (adjectives)/бродячий
824 из 957
Это исходный  /w/RNC (adjectives)/б

[]
Ready


In [3]:
#Обработка содержимого всех страниц
print('Обработка всех символов')
count=0
link_count=len(symbol_links)
texts=[]
#for link in symbol_links:
for link in links:
    count+=1
    try:
        SymbolFile = urllib.request.urlopen(get_page_url(unquote(link)))        
    except URLError as e:
        continue
    else:
        SymbolHtml = SymbolFile.read()
        SymbolFile.close()
        soup = BeautifulSoup(SymbolHtml)
        parsed=link.split('/')
        filename=parsed[len(parsed)-1]
        text=str(soup.get_text())
        with open('rucontext\\'+filename+'.txt','w',encoding='utf-8') as f:
            f.write(text)
        print('Обработано {} страниц из {}'.format(count,link_count))
print('ready')


Обработка всех символов
Обработано 3 страниц из 0
Обработано 4 страниц из 0
Обработано 5 страниц из 0
Обработано 6 страниц из 0
Обработано 7 страниц из 0
Обработано 8 страниц из 0
Обработано 9 страниц из 0
Обработано 10 страниц из 0
Обработано 11 страниц из 0
Обработано 12 страниц из 0
Обработано 13 страниц из 0
Обработано 14 страниц из 0
Обработано 15 страниц из 0
Обработано 16 страниц из 0
Обработано 17 страниц из 0
Обработано 18 страниц из 0
Обработано 19 страниц из 0
Обработано 20 страниц из 0
Обработано 21 страниц из 0
Обработано 22 страниц из 0
Обработано 23 страниц из 0
Обработано 24 страниц из 0
Обработано 25 страниц из 0
Обработано 26 страниц из 0
Обработано 27 страниц из 0
Обработано 28 страниц из 0
Обработано 29 страниц из 0
Обработано 30 страниц из 0
Обработано 31 страниц из 0
Обработано 32 страниц из 0
Обработано 33 страниц из 0
Обработано 34 страниц из 0
Обработано 35 страниц из 0
Обработано 36 страниц из 0
Обработано 37 страниц из 0
Обработано 38 страниц из 0
Обработано 

Обработано 301 страниц из 0
Обработано 302 страниц из 0
Обработано 303 страниц из 0
Обработано 304 страниц из 0
Обработано 305 страниц из 0
Обработано 306 страниц из 0
Обработано 307 страниц из 0
Обработано 308 страниц из 0
Обработано 309 страниц из 0
Обработано 310 страниц из 0
Обработано 311 страниц из 0
Обработано 312 страниц из 0
Обработано 313 страниц из 0
Обработано 314 страниц из 0
Обработано 315 страниц из 0
Обработано 316 страниц из 0
Обработано 317 страниц из 0
Обработано 318 страниц из 0
Обработано 319 страниц из 0
Обработано 320 страниц из 0
Обработано 321 страниц из 0
Обработано 322 страниц из 0
Обработано 323 страниц из 0
Обработано 324 страниц из 0
Обработано 325 страниц из 0
Обработано 326 страниц из 0
Обработано 327 страниц из 0
Обработано 328 страниц из 0
Обработано 329 страниц из 0
Обработано 330 страниц из 0
Обработано 331 страниц из 0
Обработано 332 страниц из 0
Обработано 333 страниц из 0
Обработано 334 страниц из 0
Обработано 335 страниц из 0
Обработано 336 стран

Обработано 594 страниц из 0
Обработано 595 страниц из 0
Обработано 596 страниц из 0
Обработано 597 страниц из 0
Обработано 598 страниц из 0
Обработано 599 страниц из 0
Обработано 600 страниц из 0
Обработано 601 страниц из 0
Обработано 602 страниц из 0
Обработано 603 страниц из 0
Обработано 604 страниц из 0
Обработано 605 страниц из 0
Обработано 606 страниц из 0
Обработано 607 страниц из 0
Обработано 608 страниц из 0
Обработано 609 страниц из 0
Обработано 610 страниц из 0
Обработано 611 страниц из 0
Обработано 612 страниц из 0
Обработано 613 страниц из 0
Обработано 614 страниц из 0
Обработано 615 страниц из 0
Обработано 616 страниц из 0
Обработано 617 страниц из 0
Обработано 618 страниц из 0
Обработано 619 страниц из 0
Обработано 620 страниц из 0
Обработано 621 страниц из 0
Обработано 622 страниц из 0
Обработано 623 страниц из 0
Обработано 624 страниц из 0
Обработано 625 страниц из 0
Обработано 626 страниц из 0
Обработано 627 страниц из 0
Обработано 628 страниц из 0
Обработано 629 стран

Обработано 889 страниц из 0
Обработано 890 страниц из 0
Обработано 891 страниц из 0
Обработано 892 страниц из 0
Обработано 893 страниц из 0
Обработано 894 страниц из 0
Обработано 895 страниц из 0
Обработано 896 страниц из 0
Обработано 897 страниц из 0
Обработано 898 страниц из 0
Обработано 899 страниц из 0
Обработано 900 страниц из 0
Обработано 901 страниц из 0
Обработано 902 страниц из 0
Обработано 903 страниц из 0
Обработано 904 страниц из 0
Обработано 905 страниц из 0
Обработано 906 страниц из 0
Обработано 907 страниц из 0
Обработано 908 страниц из 0
Обработано 909 страниц из 0
Обработано 910 страниц из 0
Обработано 911 страниц из 0
Обработано 912 страниц из 0
Обработано 913 страниц из 0
Обработано 914 страниц из 0
Обработано 915 страниц из 0
Обработано 916 страниц из 0
Обработано 917 страниц из 0
Обработано 918 страниц из 0
Обработано 919 страниц из 0
Обработано 920 страниц из 0
Обработано 921 страниц из 0
Обработано 922 страниц из 0
Обработано 923 страниц из 0
Обработано 924 стран

In [ ]:
for text in texts:
    print(text)

In [11]:
import rlwsd
lst = rlwsd.list_words()
#for l in lst:
    #model=rlwsd.SphericalModel.load(l)
    #?model
    #print(model.senses)
model=rlwsd.SphericalModel.load('абрикос')
model

# Работа с корпусом и эмоциями

In [251]:
import pymorphy2

from spacy.lang import ru
from nltk.stem.snowball import SnowballStemmer 
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop = set(nltk.corpus.stopwords.words('russian'))
Unused_chars=[':',',','.','-','\n','?','!']
stemmer = SnowballStemmer('russian')
morph = pymorphy2.MorphAnalyzer()


def tokenize(file_text):
    tokens = word_tokenize(file_text)
    tokens = [i for i in tokens if (i not in stop)]
    tokens = [i.replace("«", "").replace("»", "") for i in tokens]
    return tokens


def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 3]
    tokens = [get_lemma(token) for token in tokens]
    tokens = [morph.parse(token)[0].normal_form for token in tokens]
    return tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    

def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nikov\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
emotion_name=['Positive','Negative','Anger','Anticipation','Disgust','Fear','Joy','Sadness','Surprise','Trust']
Positive_Emotions=['Positive','Joy','Surprise','Trust']
Negative_Emotions=['Negative','Anger','Anticipation','Disgust','Fear','Sadness','Surprise']


In [253]:
def lyrics_file_to_token(filename):
    text=[]
    with open('Lyrics\\'+filename,encoding='utf-8') as f:
        for line in f:
            tokens = prepare_text_for_lda(line)
            for token in tokens:
                text.append(token)
    return text
Chapter2_text=lyrics_file_to_token('Chapter2.txt')
Chapter13_text=lyrics_file_to_token('Chapter13.txt')

In [255]:
import csv
count=0
emotion_arr=[0,0,0,0,0,0,0,0,0,0]
input_file = csv.DictReader(open("Docs\\bigsize_with_user_tags.csv"))
inc=0
for row in input_file:
    another=prepare_text_for_lda(row['word'])
    inc+=1
    print(inc)
    for token in Chapter2_text:
        if (len(another)==0):
            continue
        if (token==another[0]):
            for i in range(10):
                emotion_arr[i]+=int(row[emotion_name[i]])                   
print(emotion_arr)
with open('emotion_Chapter2_freq.txt','w',encoding='utf-8') as f:
    for i in range(10):
        f.write(emotion_name[i] + ' '+str(emotion_arr[i])+'\n')


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


5137
5138
5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336


8413
8414
8415
8416
8417
8418
8419
8420
8421
8422
8423
8424
8425
8426
8427
8428
8429
8430
8431
8432
8433
8434
8435
8436
8437
8438
8439
8440
8441
8442
8443
8444
8445
8446
8447
8448
8449
8450
8451
8452
8453
8454
8455
8456
8457
8458
8459
8460
8461
8462
8463
8464
8465
8466
8467
8468
8469
8470
8471
8472
8473
8474
8475
8476
8477
8478
8479
8480
8481
8482
8483
8484
8485
8486
8487
8488
8489
8490
8491
8492
8493
8494
8495
8496
8497
8498
8499
8500
8501
8502
8503
8504
8505
8506
8507
8508
8509
8510
8511
8512
8513
8514
8515
8516
8517
8518
8519
8520
8521
8522
8523
8524
8525
8526
8527
8528
8529
8530
8531
8532
8533
8534
8535
8536
8537
8538
8539
8540
8541
8542
8543
8544
8545
8546
8547
8548
8549
8550
8551
8552
8553
8554
8555
8556
8557
8558
8559
8560
8561
8562
8563
8564
8565
8566
8567
8568
8569
8570
8571
8572
8573
8574
8575
8576
8577
8578
8579
8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8601
8602
8603
8604
8605
8606
8607
8608
8609
8610
8611
8612


10043
10044
10045
10046
10047
10048
10049
10050
10051
10052
10053
10054
10055
10056
10057
10058
10059
10060
10061
10062
10063
10064
10065
10066
10067
10068
10069
10070
10071
10072
10073
10074
10075
10076
10077
10078
10079
10080
10081
10082
10083
10084
10085
10086
10087
10088
10089
10090
10091
10092
10093
10094
10095
10096
10097
10098
10099
10100
10101
10102
10103
10104
10105
10106
10107
10108
10109
10110
10111
10112
10113
10114
10115
10116
10117
10118
10119
10120
10121
10122
10123
10124
10125
10126
10127
10128
10129
10130
10131
10132
10133
10134
10135
10136
10137
10138
10139
10140
10141
10142
10143
10144
10145
10146
10147
10148
10149
10150
10151
10152
10153
10154
10155
10156
10157
10158
10159
10160
10161
10162
10163
10164
10165
10166
10167
10168
10169
10170
10171
10172
10173
10174
10175
10176
10177
10178
10179
10180
10181
10182
10183
10184
10185
10186
10187
10188
10189
10190
10191
10192
10193
10194
10195
10196
10197
10198
10199
10200
10201
10202
10203
10204
10205
10206
10207
10208
1020

12773
12774
12775
12776
12777
12778
12779
12780
12781
12782
12783
12784
12785
12786
12787
12788
12789
12790
12791
12792
12793
12794
12795
12796
12797
12798
12799
12800
12801
12802
12803
12804
12805
12806
12807
12808
12809
12810
12811
12812
12813
12814
12815
12816
12817
12818
12819
12820
12821
12822
12823
12824
12825
12826
12827
12828
12829
12830
12831
12832
12833
12834
12835
12836
12837
12838
12839
12840
12841
12842
12843
12844
12845
12846
12847
12848
12849
12850
12851
12852
12853
12854
12855
12856
12857
12858
12859
12860
12861
12862
12863
12864
12865
12866
12867
12868
12869
12870
12871
12872
12873
12874
12875
12876
12877
12878
12879
12880
12881
12882
12883
12884
12885
12886
12887
12888
12889
12890
12891
12892
12893
12894
12895
12896
12897
12898
12899
12900
12901
12902
12903
12904
12905
12906
12907
12908
12909
12910
12911
12912
12913
12914
12915
12916
12917
12918
12919
12920
12921
12922
12923
12924
12925
12926
12927
12928
12929
12930
12931
12932
12933
12934
12935
12936
12937
12938
1293

In [256]:
import csv
count=0
emotion_arr=[0,0,0,0,0,0,0,0,0,0]
input_file = csv.DictReader(open("Docs\\bigsize_with_user_tags.csv"))
inc=0
for row in input_file:
    inc+=1
    print(inc)
    for token in Chapter13_text:
        another=prepare_text_for_lda(row['word'])
        if (len(another)==0):
            continue
        if (token==another[0]):
            for i in range(10):
                emotion_arr[i]+=int(row[emotion_name[i]])                   
print(emotion_arr)
with open('emotion_Chapter13_freq.txt','w',encoding='utf-8') as f:
    for i in range(10):
        f.write(emotion_name[i] + ' '+str(emotion_arr[i])+'\n')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


5137
5138
5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336


8413
8414
8415
8416
8417
8418
8419
8420
8421
8422
8423
8424
8425
8426
8427
8428
8429
8430
8431
8432
8433
8434
8435
8436
8437
8438
8439
8440
8441
8442
8443
8444
8445
8446
8447
8448
8449
8450
8451
8452
8453
8454
8455
8456
8457
8458
8459
8460
8461
8462
8463
8464
8465
8466
8467
8468
8469
8470
8471
8472
8473
8474
8475
8476
8477
8478
8479
8480
8481
8482
8483
8484
8485
8486
8487
8488
8489
8490
8491
8492
8493
8494
8495
8496
8497
8498
8499
8500
8501
8502
8503
8504
8505
8506
8507
8508
8509
8510
8511
8512
8513
8514
8515
8516
8517
8518
8519
8520
8521
8522
8523
8524
8525
8526
8527
8528
8529
8530
8531
8532
8533
8534
8535
8536
8537
8538
8539
8540
8541
8542
8543
8544
8545
8546
8547
8548
8549
8550
8551
8552
8553
8554
8555
8556
8557
8558
8559
8560
8561
8562
8563
8564
8565
8566
8567
8568
8569
8570
8571
8572
8573
8574
8575
8576
8577
8578
8579
8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8601
8602
8603
8604
8605
8606
8607
8608
8609
8610
8611
8612


10043
10044
10045
10046
10047
10048
10049
10050
10051
10052
10053
10054
10055
10056
10057
10058
10059
10060
10061
10062
10063
10064
10065
10066
10067
10068
10069
10070
10071
10072
10073
10074
10075
10076
10077
10078
10079
10080
10081
10082
10083
10084
10085
10086
10087
10088
10089
10090
10091
10092
10093
10094
10095
10096
10097
10098
10099
10100
10101
10102
10103
10104
10105
10106
10107
10108
10109
10110
10111
10112
10113
10114
10115
10116
10117
10118
10119
10120
10121
10122
10123
10124
10125
10126
10127
10128
10129
10130
10131
10132
10133
10134
10135
10136
10137
10138
10139
10140
10141
10142
10143
10144
10145
10146
10147
10148
10149
10150
10151
10152
10153
10154
10155
10156
10157
10158
10159
10160
10161
10162
10163
10164
10165
10166
10167
10168
10169
10170
10171
10172
10173
10174
10175
10176
10177
10178
10179
10180
10181
10182
10183
10184
10185
10186
10187
10188
10189
10190
10191
10192
10193
10194
10195
10196
10197
10198
10199
10200
10201
10202
10203
10204
10205
10206
10207
10208
1020

12773
12774
12775
12776
12777
12778
12779
12780
12781
12782
12783
12784
12785
12786
12787
12788
12789
12790
12791
12792
12793
12794
12795
12796
12797
12798
12799
12800
12801
12802
12803
12804
12805
12806
12807
12808
12809
12810
12811
12812
12813
12814
12815
12816
12817
12818
12819
12820
12821
12822
12823
12824
12825
12826
12827
12828
12829
12830
12831
12832
12833
12834
12835
12836
12837
12838
12839
12840
12841
12842
12843
12844
12845
12846
12847
12848
12849
12850
12851
12852
12853
12854
12855
12856
12857
12858
12859
12860
12861
12862
12863
12864
12865
12866
12867
12868
12869
12870
12871
12872
12873
12874
12875
12876
12877
12878
12879
12880
12881
12882
12883
12884
12885
12886
12887
12888
12889
12890
12891
12892
12893
12894
12895
12896
12897
12898
12899
12900
12901
12902
12903
12904
12905
12906
12907
12908
12909
12910
12911
12912
12913
12914
12915
12916
12917
12918
12919
12920
12921
12922
12923
12924
12925
12926
12927
12928
12929
12930
12931
12932
12933
12934
12935
12936
12937
12938
1293

In [267]:
Chapter2_text=[]
with open('Lyrics\Chapter2.txt',encoding='utf-8') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        Chapter2_text.append(tokens)

In [270]:
import csv
#import pandas as pd
from polyglot.text import Text as T
from polyglot.detect import Detector

count=0
emotion_arr=[0,0,0,0,0,0,0,0,0,0]
input_file=[]
with open("Docs\\bigsize_with_user_tags.csv") as csvfile:
    input_file1 = csv.DictReader(csvfile,delimiter=",")
    input_file=list(input_file1)
inc=0
temp=0

In [272]:
little_text1=''
little_text2=''
for tokens in Chapter2_text:
    little_text1=''
    for token in tokens:
        little_text+=token+' '
    poly_text=T(little_text)
    for token in tokens:
        inc+=1
        print(inc)
        for row in input_file:
            another=prepare_text_for_lda(row['word'])
            if (len(another)==0):
                continue
            if (token==another[0]):
                if(poly_text.polarity>0):
                    for i in range(10):
                        if(emotion_name[i] in Positive_Emotions):
                            emotion_arr[i]+=int(row[emotion_name[i]])
                else:
                    for i in range(10):
                        if(emotion_name[i] in Negative_Emotions):
                            emotion_arr[i]+=int(row[emotion_name[i]])
                break
print(emotion_arr)
with open('emotion_Chapter2_freq.txt','w',encoding='utf-8') as f:
    for i in range(10):
        f.write(emotion_name[i] + ' '+str(emotion_arr[i])+'\n')

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138


ValueError: Package 'sentiment2.un' not found in index

-1.0


In [292]:
import csv
Lem_Dict=['он','она','варенуха','вампир','ленинград','варьете','римский','лиходеев','телеграмма','ялта','обещание','сатана','гелла','азазелло','воланд','свита','дождь','фагот','квартира','дождь','бал','берлиоз','смерть','гора','крест','иисус','иуда','голова','пилат','бездомный','сон','каиф','бал','валюта','деньги','бумага','босой','афраний','мастер','собака','вар-равван','крысобой','вино','критика','покой','рукопись','матвей','огонь','маргарита','гость','мазь','брак','королева','голый']
Lem_Dict_Verb=['быть','любить','обретать','пить','отдать','писать','выиграть','разгромить','оказаться','подбросить','созвать','получать','получить','иметь','вести','снять','давать','дать','болеть','обрушиться','нести','слать','искать','оказаться','выполнить','отправить','сдать','приветствовать','стать','использовать','гореть','не гореть','унести','пить','обрести','предсказать','описать','принять','обрести','вылечить','называться','бить','убить','любить','допрашивать','принять','записывать','поручить','читать','видеть','писать','отдать','обрести']
Lemmed_Dict=[]
Lemmed_Dict_Verb=[]
for nyan in Lem_Dict:
    Lemmed_Dict.append(prepare_text_for_lda(nyan))
for nyan in Lem_Dict_Verb:
    temp=prepare_text_for_lda(nyan)
    if temp not in Lemmed_Dict_Verb and temp!='':
        Lemmed_Dict_Verb.append(temp)
    

In [2]:
#print(Lemmed_Dict)
#Lemmed_Dict_Verb.pop(0)
#print(Lemmed_Dict_Verb)

0


In [286]:
Little_sema_list_noun=[]
Little_sema_list_verb=[]
with open("Docs\\bigsize_with_user_tags.csv") as csvfile:
    input_file = csv.DictReader(csvfile,delimiter=",")
    for row in input_file:
        row_lemmed=prepare_text_for_lda(row['word'])
        if row_lemmed in Lemmed_Dict and len(row_lemmed)>0:
            Little_sema_list_noun.append(row)
        elif row_lemmed in Lemmed_Dict_Verb and len(row_lemmed)>0:
            Little_sema_list_verb.append(row)
        else:
            continue
print('ready')

ready


In [291]:
from polyglot.text import Text as T
from polyglot.detect import Detector

triple_string=[]
with open('Lyrics\Master.txt',encoding='utf-8') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        triple_string.append(tokens)
        if len(triple_string)>3:
            triple_string.pop(0)
        for token in tokens:
            if token in Lemmed_Dict:
                for noun in Lem_Dict:
                    for string in triple_string:
                        if noun in string and noun!=token:
                            print('checked')
                                

